# Introduction to Neural Networks

Neural networks are a very flexible model through which to learn complex, high dimensional functions. First, we will apply the model in Pytorch, and then implement it by hand, using multivariable calculus to derive the parameter updates.

In this example, we will apply a neural network to an image classification problem. We have a dataset containing images clothes, with each image having a label describing the item of clothing in the image. Each $28 \times 28$-pixel image is represented as a length $784$ vector $\mathbf{x}$. We use bold font to indicate a vector. There are 10 possible items of clothing (e.g. t-shirt, hat) in our dataset, so $y$ is an integer in the range $[0, 10]$. Together, our sample space is pairings of images and labels, $(\mathbf{x}_i, y_i) \sim (\mathcal{X}, \mathcal{Y})$ (i.e. a single image, label pair index by $i$ can be drawn from the sample space of all pairs of images and labels). s  


We want to learn the conditional distribution $p(y| \mathbf{x})$ i.e. what is the probability of the label $y$ given an image, $\mathbf{x}$. For example, what is the probability this is an image $\mathbf{x}$ is of a t-shirt? In this case, we want to learn a function $f(\mathbf{x}) : \mathbf{x} \rightarrow \mathbf{y}$. It will take a vector of length $784$ and output a vector of length $10$, with each element of the output vector assigning some weight related to the probability of image $\mathbf{x}$ being a particular label $y$. These unnormalised weights output by the model are called 'logits'.

<img src="nn-image-1.png" width="400">

In this example, we will first follow the [Pytorch](https://docs.pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html) 'Getting Started' page to train a simple model for this classification task. Then, we will get a deeper grip on how these models work by implementing the model ourselves in Numpy, which requires calculating the derivatives required for model training ourselves.

## Coding up a Neural Network in Pytorch

This section exactly follows [the Pytorch introduction](https://docs.pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html).

### The data

We will use the FashionMINST dataset, containing $28 \times 28$ images of clothing, 10 possible labels. In total there are $60,000$ image sin the training set and $10,000$ images in the test set.

<img src="FashionMNIST.png" width="500">

In [ ]:
import torch
from torch import nn
# DataLoader is an iterable around DataSet, which stores samples and their corresponding labels.
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",  # root directory
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


In [ ]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


This makes sense, our batch is (batch size, num channels (RGB), image height, image width) and for each image in the batch we have a single label (e.g. X = t-shirt).

When training, Batch Size...

Epoch...

### The Model

The very cool thing, we can just define the inputs, outputs, and define an architecture that we will be able to mould into the function of interest. Note the amazing beauty of how flexible this is, because of our set up. We simply define inputs, outputs and a black-box architecture (for now). Then we don't need to think about whats going on inside, we 'shape' this through iterative training of the weights on our known data. Once we have got a good mould we are done!


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()

        # Linear layers are:  yᵢ = φ( ∑ⱼ xⱼ Wᵀⱼᵢ + bᵢ ) so together y = φ(xW^T + b)
        # where W is a (output_features, input_features) set of weights, b is vector of biases
        # so we can easily control output shape. Layers are fully connected.
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)




Using cpu
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)



#### What is the Model Learning?

Let's just look again what the model is learning. To fully characterise the dataset,  we would have the joint distribution
between the data and the labels p(x, y). We can draw this out in the simple case:


This also gives us information about the relative frequencies of the labels, y and images. Then we could convert to p(y|X) by dividing by p(x) i.e. Bayes


However, this is much more complicated, and the aim of generative models. This is what we need to learn for generative models! but, it is more complex.
Can maginalise out

In our case, we will take a shortcut and just learn p(y|x). Basically it learns the feature space and how that maps to 28*28 dimension space and partitions it
into 10 classes. The decision boundary is a hyper-surface in the space. Of course, this tells us nothing about p(x, y) because XXX. but it is a nice way of p(y|X).

A simple drawing!

but in reality, it is not a simple plane but a manifold in much hig

hmmm, still not really sure what exactly we are learning. DOes this literally funnel an unkonwn x into a already-known x and then map p(y|x)?

https://arxiv.org/abs/1311.2901
http://neuralnetworksanddeeplearning.com/
https://cs231n.github.io/linear-classify/
https://ai.stanford.edu/~ang/papers/nips01-discriminativegenerative.pdf

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# We can see we have Weights, Biases, Weights, Biases, Weights, Biases (3 layers)
print(f"The model parameters")
for name, param in model.named_parameters():
      print(
          f"Name: {name}\n"
          f"Type: {type(param)}\n"
          f"Size: {param.size()}\n"
          f"e.g. {param[:5]}\n\n")

The model parameters
Name: linear_relu_stack.0.weight
Type: <class 'torch.nn.parameter.Parameter'>
Size: torch.Size([512, 784])
e.g. tensor([[ 0.0242, -0.0248, -0.0166,  ..., -0.0256,  0.0325, -0.0028],
        [ 0.0264,  0.0106, -0.0349,  ..., -0.0113, -0.0121,  0.0221],
        [ 0.0038, -0.0243,  0.0197,  ..., -0.0243, -0.0306,  0.0113],
        [-0.0337, -0.0340, -0.0342,  ...,  0.0348, -0.0187, -0.0099],
        [ 0.0065,  0.0300,  0.0117,  ...,  0.0249, -0.0121,  0.0218]],
       grad_fn=<SliceBackward0>)


Name: linear_relu_stack.0.bias
Type: <class 'torch.nn.parameter.Parameter'>
Size: torch.Size([512])
e.g. tensor([-0.0340, -0.0322, -0.0116,  0.0172, -0.0129], grad_fn=<SliceBackward0>)


Name: linear_relu_stack.2.weight
Type: <class 'torch.nn.parameter.Parameter'>
Size: torch.Size([512, 512])
e.g. tensor([[ 0.0256, -0.0107, -0.0299,  ..., -0.0403,  0.0318, -0.0249],
        [ 0.0342, -0.0205,  0.0428,  ...,  0.0127,  0.0360, -0.0173],
        [-0.0149,  0.0350, -0.0221,  ..., 

### Training and Evaluating the Model

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train(mode=True)  # put into 'training mode'

    for batch, (X, y) in enumerate(dataloader):

        X, y = X.to(device), y.to(device)

        # Pred is (64, 10) tuple of predictions for this batch
        # y is (64, 1) (classes)
        # Cross entropy loss https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()  # perform one step θt <- f(θ_{t-1})
        optimizer.zero_grad()  # zero the accumulated gradients, ready for the next step

        if batch % 100 == 0:
            loss = loss.item()  # Note `loss` is a object, we use `item()` to get the scalar loss
            current = (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

train(train_dataloader, model, loss_fn, optimizer)

loss: 2.311416  [   64/60000]
loss: 2.297227  [ 6464/60000]
loss: 2.284765  [12864/60000]
loss: 2.277797  [19264/60000]
loss: 2.245504  [25664/60000]
loss: 2.223144  [32064/60000]
loss: 2.229235  [38464/60000]
loss: 2.200769  [44864/60000]
loss: 2.204567  [51264/60000]
loss: 2.150434  [57664/60000]


In [ ]:
def test(dataloader, model, loss_fn):
    """"""
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()  # Go from train to eval mode

    test_loss, correct = 0, 0

    with torch.no_grad():  # this just turns of gradient computation for speed

        for batch, (X, y) in enumerate(dataloader):

            X, y = X.to(device), y.to(device)

            pred = model(X)
            # pred_i = torch.argmax(torch.exp(pred) / torch.sum(torch.exp(pred)), axis=1)
            pred_i = pred.argmax(1)  # of course, it doesn't matter if the logits are passed through softmax, which maintains transitivity
            correct += (pred_i == y).type(torch.float).sum().item()
            test_loss += loss_fn(pred, y)

        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

print("Epoch 1")
test(test_dataloader, model, loss_fn)

for i in range(5):
    print(f"Epoch {i + 2}")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)


Epoch 1
Test Error: 
 Accuracy: 36.9%, Avg loss: 2.159129 

Epoch 2
loss: 2.175913  [   64/60000]
loss: 2.156889  [ 6464/60000]
loss: 2.109339  [12864/60000]
loss: 2.123337  [19264/60000]
loss: 2.051893  [25664/60000]
loss: 2.000705  [32064/60000]
loss: 2.027452  [38464/60000]
loss: 1.956121  [44864/60000]
loss: 1.970606  [51264/60000]
loss: 1.868530  [57664/60000]
Test Error: 
 Accuracy: 49.2%, Avg loss: 1.889209 

Epoch 3
loss: 1.925272  [   64/60000]
loss: 1.883625  [ 6464/60000]
loss: 1.786613  [12864/60000]
loss: 1.827395  [19264/60000]
loss: 1.697302  [25664/60000]
loss: 1.655534  [32064/60000]
loss: 1.676771  [38464/60000]
loss: 1.589110  [44864/60000]
loss: 1.623531  [51264/60000]
loss: 1.493784  [57664/60000]
Test Error: 
 Accuracy: 62.1%, Avg loss: 1.532667 

Epoch 4
loss: 1.596773  [   64/60000]
loss: 1.552969  [ 6464/60000]
loss: 1.422226  [12864/60000]
loss: 1.491803  [19264/60000]
loss: 1.362407  [25664/60000]
loss: 1.363337  [32064/60000]
loss: 1.370365  [38464/60000]
lo

Add a note here on generative vs discrimiantive. Here we only learn p(y|x) NOT p(x, y)! Here we simply do ML on p(y|x) !!!!



## Training a Neural Network by Hand - A 'Simple' Example

Now, we will implement the same model that we created in Python, but this time 'by-hand' in Numpy. To do this, we will have  to calculate the derivatives of the parameters with respect to the loss function, in order to update them during training. 
First, we will review a simplified version of the model that contains only weights, but no bias or nonlinear functions.

### The Model

Let $l^1$, $l^2$ and $l^3$ be vector-valued functions representing the layers $1$, $2$, $3$ of the network. $l^1$ takes our length $784$ row vector $\mathbf{x}$ and returns a length $512$ vector. $l^2$ takes this length $512$ vector as input and returns another length $512$ veector. Our final layer $l^3$ takes this a length $512$ vector as input and outputs a length $10$ vector. 
\begin{aligned}
    &l^1(\mathbf{x}) = \mathbf{x}W^1 \ \ \ \ \text{(1, 784) x (784, 512) = (1, 512)} \\
    &l^2(\mathbf{l^1}) = \mathbf{l^1}W^2 \ \ \ \ \text{(1, 512) x (512, 512) = (1, 512)} \\
    &l^3(\mathbf{l^2}) = \mathbf{l^2}W^3 \ \ \ \ \text{(1, 512) x (512, 10) = (1, 10)} \\
\end{aligned}

All vectors are row vectors. The $W$ are our matrices of weights with shape (input, output): $W^1$ is $(10, 512)$,  $W^2$ is $(512, 512)$ and $W^3$ is $(512, 10)$. We can therefore index individual weights as $W^3_{i, o}$ where $i$ is the index of the input unit (i.e. the unit the weight connects from), and $o$ is the index of the output unit (i.e. the unit the weight connects to). 

You can see by **writing out the matrix multiplication** that this operation matches the information flow through the network.

[IMAGE]

*(An aside)*: This notation a little non-standard (often vectors are column vectors, and the weight matrix is shape (output, input) but this notation makes the derivations below much simpler. Also the layers are functions $l^3(\cdot)$ but can be treated as vectors $\mathbf{l^3}$ when evaluated. We will typically write them as vectors.

### The Loss

**TODO**: add, define, use the word 'score'

We take the $(1, 10)$ shape vector of logits (representing score for each label) output from the final layer, and use these to compute our loss. We will use the cross-entropy loss:

$$
L(\mathbf{l^3}, y) = -\log \dfrac{ \exp{ l^3_y }}{ \sum_k \exp{ l^3_k }}
$$

Here $l^3_k$ is the element of our vector of logits with some index $k$, and $y$ is the index of the correct label for this image. You may recognise the term after the $\log$ as the [softmax function](https://en.wikipedia.org/wiki/Softmax_function#:~:text=The%20softmax%20function%2C%20also%20known,used%20in%20multinomial%20logistic%20regression) which normalises the logits to probabilities. Therefore, we are computing the probability our model assigns the input image $\mathbf{x}$ being (the correct) label $y$. Clearly, we want this probability to be high ($1$, ideally). [This article](https://cs231n.github.io/linear-classify/) as a nice, deeper discussion of the Cross Entropy Loss.

So this loss makes intuitive sense. We have an image $\mathbf{x}$ and are computing a set of probabilities, one for each of the $10$ labels. We are aiming to maximise the probability associated with the correct label, $y$, as we know in this supervised learning context that this is the actual label for this image. Here all we are doing on top is (equivalently) minimising the negative log probability.


### Predicting a label from an image - the forward pass

For this simple network, it is reliatively easy to use our model to i) take an image $\mathbf{x}$ ii)  map it to our output of 10 logits iii) use these to predict a label $\hat{y}$. First we apply the network to the input data:

\begin{aligned}
\mathbf{l^3} &= l^3(l^2(l^1(\mathbf{x}))) \\
             &= (((\mathbf{x}W^1) W^2) W^3 \\
             &=  \mathbf{x}W^1 W^2 W^3 \\
\end{aligned}

We then take this output, and the predicted label is the one that maximises the probability as copmuted by the softmax function:

$$
\hat{y} = \arg\max_{y} \, \mathrm{softmax}(\mathbf{l^3})_y
$$

In other words, the predicted label is the one the one the model assigns the highest probability to. The notation can be confusing at first look, but we take advantage of that fact that the labels are defined as indices—so we can use the correct label $y$ to index out the corresponding entry in the vector of probaiblities.

### Backpropagation in our simple network

We want to find the set of weights that minimise the loss function. To do this, we will traverse this function by travelling along it's negative slope. Mathematically, this means we need to compute the derivative of our loss function with respect to the weights.

For example, take $W^3_{1,2}$ that connects the first neuron from layer 2 to the second neuron in layer 3. How does a small change in this weight change the output of our loss function? In this case, a small change in $W^3_{1,2}$ will result in a small change in the second neuron in layer 3 ($l^3_2$) which will directly affect the loss function $L(\mathbf{l^3})$. This 'chain' of dependencies is capture by chain rule. Here we use [partial derivatives](https://en.wikipedia.org/wiki/Partial_derivative) because we only care about how a function that takes mutliple inputs changes with respect to a single one of those inputs.

\begin{aligned}
&\dfrac{\partial L(\mathbf{l^3})}{\partial W^3_{1,2}} = \dfrac{\partial L(\mathbf{l^3})}{\partial l^3_2}  \dfrac{\partial l^3_2 }{\partial W^3_{1,2}}
\end{aligned}

Often, we will write equations that collect all weights for a layer and their affect on all neurons in a layer. Also, from now on, we will write the loss as $L$ instead of $L(\mathbf{l^3})$ just for brevity, but it is useful to remember it a function of out output layer. Together:

\begin{aligned}
&\dfrac{\partial L}{\partial W^3} = \dfrac{\partial L}{\partial \mathbf{l^3}}   \dfrac{\partial \mathbf{l^3} }{\partial W^3} 
\end{aligned}

and for the other weight matrices:

\begin{aligned}
&\dfrac{\partial L}{\partial W^2} = \dfrac{\partial L}{\partial \mathbf{l^3}}  \dfrac{\partial \mathbf{l^3}}{\partial \mathbf{l^2}}  \dfrac{\partial \mathbf{l^2} }{\partial W^3}  \\
&\dfrac{\partial L}{\partial W^1} = \dfrac{\partial L}{\partial \mathbf{l^3}}  \dfrac{\partial \mathbf{l^3}}{\partial \mathbf{l^2}}  \dfrac{\partial \mathbf{l^2}}{\partial \mathbf{l^1}}  \dfrac{\partial \mathbf{l^1} }{\partial W^1}
\end{aligned}

TODO: CHECK AGAINST https://www.jasonosajima.com/backprop.html

e.g. for how the loss changes with a change in weights in layer 2, we look at: i) how changing the weights in layer 2 affects the output layer 2 ii) how a change in layer 2 affects the output of layer 3 ii) how a change in layer 3 affects the loss.

#### Understanding each term with [matrix calculus](https://en.wikipedia.org/wiki/Matrix_calculus)

This notation is doing a lot of heavy lifting and hiding complexity. Take $\dfrac{\partial L}{\partial W^2}$, $L$ is a scalar-valued function (it takes as an input a vector of length 10, our output of layer 3, and returns a scalar). We want to take the deriative of this with respect to a matrix—how each element of $W^3_{i,o}$ affects the loss. Therefore this derivative will be a $(512, 10)$  matrix (the size of $W^3$). 

How about $\dfrac{\partial \mathbf{l^2}}{\partial \mathbf{l^3}}$? This is the derivative of a vector-valued function (it outputs a vector, of length $512$) with repsect to a vector! This is the Jacobian, which tracks how element $l^2$ affects each element in  $l^3$. As layer 3 has $10$ neurons and layer 2 has $512$, we will have a $(10, 512)$ matrix of partial derivatives.

$\dfrac{\partial \mathbf{l^2} }{\partial W^3}$ asks how a small change in each weight in $W^2$ will affect each dimension in $\mathbf{l^2}$. We will have a $(512, 512, 512)$, rank-3 tensor!

#### Computing the derivatives with respect to $W^3$

Fortunately the structure of our network means many of these partial derivatives are zero, and we can simply this a lot. Below, we go through each of these in turn, before implementing this network in Python. First we will look in detail at computing the derivative of the loss with respect to the weights connected to the final layer:

$$
\dfrac{\partial L}{\partial W^3} = \dfrac{\partial L}{\partial \mathbf{l^3}}   \dfrac{\partial \mathbf{l^3} }{\partial W^3} 
$$

##### $\dfrac{\partial L}{\partial W^3}$

This is the exact derivative we want to compute to update the weights for the third layer, $W^3$. It will be a $(512, 10)$ matrix, with each element being how a small change in that weight affects the loss e.g. (we drop the superscript for brevity):

$$
    \dfrac{\partial L }{\partial W} =
    \begin{bmatrix}
        \dfrac{ \partial L }{ \partial W_{1,1} } & \dfrac{ \partial L }{ \partial W_{1,2} } & ... & \dfrac{ \partial L }{ \partial W_{1,10} }\\
        \dfrac{ \partial L }{ \partial W_{2,1} } & \ddots & & \vdots \\
        \vdots \\
        \dfrac{ \partial L }{ \partial W_{512,1} } & \dots & &   \dfrac{ \partial L }{ \partial W_{512,10} }
    \end{bmatrix}
$$

Next we will explore how to compute the elements of this matrix.

##### $\dfrac{\partial L}{\partial \mathbf{l^3}}$

This is the derivative of a scalar-valued function with respect to a vector. In our case, it will be a 10-dimension vector of partial derivatives, where each entry captures the loss changes with respect to a change in that unit of layer 3. 

We can evaluate the derivative of the cross entropy loss directly:

$$
\dfrac{\partial }{\partial l^3_k} - \log \dfrac{e^{l^3_y}}{\sum_k e^{l^3_k}} = \text{softmax}(l^3_k) - \delta_{k, y}
$$

 
Where $\delta$ is the [Kronecker delta](https://en.wikipedia.org/wiki/Kronecker_delta) that equals $1$ when $k=y$ and $0$ otherwise. See **Appendix 1** for the derivation. In full this looks like:

$$
\dfrac{ \partial L}{ \partial \mathbf{l^3} } =
\begin{bmatrix}
    \dfrac{\partial L}{\partial l^3_1} \\
    \dfrac{\partial L}{\partial l^3_2} \\
    \vdots\\
    \dfrac{\partial L}{\partial l^3_{10}}
\end{bmatrix}
=
\begin{bmatrix}
    \text{softmax}(l^3_1) - \delta_{1, y} \\
    \text{softmax}(l^3_2) - \delta_{2, y} \\
    \vdots \\
    \text{softmax}(l^3_{10}) - \delta_{10, y}
\end{bmatrix}
$$


Together, this says that our vector of derivatives is $\text{softmax}(l^3_i)$ for all output logits in layer 3 that are for the incorrect labels, and $\text{softmax}(l^3_i) - 1$ for the index of the true label $y$. 


##### $\dfrac{\partial \mathbf{l^3} }{\partial W^3}$

As mentioned above, this is a 3-rank tensor with the shape $(512, 10, 10)$ (how each element of $l^3_n$ changes with each weight $W^3_{i,o}$). How can we even deal with this in our computation?

Luckily, the structure of our layered neural network means that this can be significantly reduced, as most of the elements are zero. The strategy is to break the problem down to look at individual elements of $\dfrac{\partial L}{\partial W^3_{i,o}}$ and fill in each term-wise using the total derivative rule (**Appendix 2**):

\begin{align}
\dfrac{\partial L}{\partial W^3_{i,o}}
&= \dfrac{\partial L}{\partial l^3} \dfrac{\partial l^3}{\partial W^3_{i,o}}  \\
&= \sum_k \dfrac{\partial L}{\partial l^3_k} \dfrac{\partial l^3_k}{\partial W^3_{i,o}} \\
&= \dfrac{\partial L}{\partial l^3_o} \dfrac{\partial l^3_o}{\partial W^3_{i,o}} \\
&= \dfrac{\partial L}{\partial l^3_o} l^2_i
\end{align}


Obviously, the notation can get a little tricky to follow here. In words, to find out how the loss chanegs with respect to a single weight $W^3_{i,o}$ we will see how changing this weight changes layer 3, and then how changing layer 3 changes the loss. To compute this, we will take the sum over how each weight $W^3_{i,o}$ affects the unit $k$ in layer 3, $l^3_k$, then how a change in this unit affects the loss. This weight only connects to one unit, $o$ and so the effect of changing that weight on all other units is zero. 

Finally, we can take the usual scalar derivative here $\dfrac{\partial l^3_o}{\partial W^3_{i,o}} = \dfrac{\partial \ l^2_i W^3_{i,o}}{\partial W^3_{i,o}} = l^2_i$. This makes intuitive sense, the change in $l^3_o$ when we change $W^3_{i,o}$ is exactly the value of the input unit, $l^2_i$.


##### Putting this all together: $\dfrac{\partial L}{\partial W^3} = \dfrac{\partial L}{\partial \mathbf{l^3}} \dfrac{\partial \mathbf{l^3} }{\partial W^3}$

Now we can put the element-wise approach and all derivatives computed above together in one matrix. All the weights are all $W^3$ matrix and we omit the superscript for brevity:

$$
\dfrac{ \partial L}{ \partial W^3} =
\begin{bmatrix}
\dfrac{\partial L}{ \partial l^3_1 } \dfrac{\partial l^3_1 }{ \partial W_{1,1} } 
& \dfrac{\partial L}{ \partial l^3_2 } \dfrac{\partial l^3_2 }{ \partial W_{1,2} } 
& \dots 
& \dfrac{\partial L}{ \partial l^3_{10} } \dfrac{\partial l^3_{10} }{ \partial W_{1, 10} }  \\
\dfrac{\partial L}{ \partial l^3_1 } \dfrac{\partial l^3_1 }{ \partial W_{2,1} } 
& \ddots \\
\vdots \\
\dfrac{\partial L}{ \partial l^3_1 } \dfrac{\partial l^3_1 }{ \partial W_{512,1} } 
& \dots & 
& \dfrac{\partial L}{ \partial l^3_{10} } \dfrac{\partial l^3_{10} }{ \partial W_{512,10} }
\end{bmatrix}
=
\begin{bmatrix}
[\text{sm}(l^3_1) - \delta_{1, y}]l^2_1 & 
[\text{sm}(l^3_2) - \delta_{2, y}] l^2_1
& \dots 
& [\text{sm}(l^3_{10}) - \delta_{10, y}] l^2_1 & \\
[\text{sm}(l^3_1) - \delta_{1, y}] l^2_2 & \ddots & & \vdots \\
\vdots \\
[\text{sm}(l^3_1) - \delta_{1, y}] l^2_{512} & \dots & 
& [\text{sm}(l^3_{10}) - \delta_{10, y}] l^2_{512} 
\end{bmatrix}
$$

i.e. for each weight, we see the effect of changing that weight on the layer 3 unit it is connected to, multiplied by the 
effect of changing that layer 3 unit on the loss.

We can write all of this as the product of two vectors (recall all vectors are row vectors here, so the transpose is to a column vector):


$$
\dfrac{\partial L}{\partial W^3} = (\mathbf{l^2})^T \dfrac{\partial L}{\partial \mathbf{l^3}}
$$


#### Computing the derivatives with respect to $W^2$

For the most part, computing the derivative of the loss with respect to the layer 2 weights uses all of the same ideas and tricks as above. There is one element of added complexity—changing an element $W^2_{i,o}$ affects only one element of layer 2, $l^2_o$. But this layer is connected to *every* unit in layer 3, and we need to account for all of these changes. Luckily, we can iteratively compute the first terms to simpify them, so when we are ready to deal with the weights we don't need to think about all of this complexity. Recall we are trying to compute:

$$
\dfrac{\partial L}{\partial W^2} = \dfrac{\partial L}{\partial \mathbf{l^3}}  \dfrac{\partial \mathbf{l^3}}{\partial \mathbf{l^2}}  \dfrac{\partial \mathbf{l^2} }{\partial W^3}
$$

We have seen similar terms to all of these above, except for the middle term on the right hand side, the Jacobian.


##### $\dfrac{\partial \mathbf{l^2}}{\partial \mathbf{l^3}}$

This is the Jacobian. It is $(512, 10)$ and contains the derivative of each element in $\mathbf{l^2}$ with respect to each element in $\mathbf{l^3}$:

TODO: CHECK WHAT WAY AROUND THIS SHOULD BE, ALSO CHECK DIMS OF ABOUVE ARE (512, 512, 10) AND IT MATCHES THIS

\begin{aligned}
\dfrac{\partial \mathbf{l^3}}{\partial \mathbf{l^2}} = 
\begin{bmatrix}
    \dfrac{\partial l^3_1}{\partial l^2_1} & \dfrac{\partial l^3_2}{\partial l^2_1} & \dots & \dfrac{\partial l^3_{10}}{\partial l^2_1} \\
    \dfrac{\partial l^3_1}{\partial l^2_2} & \ddots & & \vdots \\
    \vdots & & \\
    \dfrac{\partial l^3_1}{\partial l^2_{512}} & \dots & & \dfrac{\partial l^3_{10}}{\partial l^2_{512}} 
\end{bmatrix}
\end{aligned}



If we try and calculate the right hand side of the above equation, we are trying to multiply a $(512,10)$ matrix $\dfrac{\partial \mathbf{l^3}}{\partial \mathbf{l^2}}$ with a $(510, 510, 10)$ 3-rank tensor $\dfrac{\partial \mathbf{l^2} }{\partial W^3}$. Luckily, we will can go from the left and collapse the Jacobian, very similar to how we collapsed the weight matrix above. As long as we compute derivatives with respect to the scalar loss, they will keep the same of the vector or matrix we are taking the derivative with respect to.

##### Putting the Layer 2 weights calculation together: 

$$
\dfrac{\partial L}{\partial W^2} = \dfrac{\partial L}{\partial \mathbf{l^3}}  \dfrac{\partial \mathbf{l^3}}{\partial \mathbf{l^2}}  \dfrac{\partial \mathbf{l^2} }{\partial W^3}
$$

First, we will calculate $\dfrac{\partial L}{\partial \mathbf{l^3}}  \dfrac{\partial \mathbf{l^3}}{\partial \mathbf{l^2}}$, that we know will be a length $512$ vector, using the total derivative rule as above:

$$
\dfrac{ \partial L }{ \partial l^2_i } = \sum_o \dfrac{ \partial L }{ \partial l^3_o } \dfrac{\partial l^3_o }{ \partial l^2_i }
$$


Note this is slightly more complex than the $W^3$ case. Because every  unit in layer 2 is connected to every unit in layer 3, the derivatives don't go to zero. We will actually need to compute every derivative. However, we can see each term in the sum only requires application of scalar derivative rules, e.g.:


$$
\dfrac{\partial l^3_o }{ \partial l^2_i } = \dfrac{\partial l^2_i W^3_{i, o} }{ \partial l^2_i } = W^3_{i, o}
$$
 XXX

So together, the full vector of partial derivatives is:

\begin{aligned}
\begin{bmatrix}
    \sum_o \dfrac{ \partial L }{ \partial l^3_o } \dfrac{\partial l^3_o }{ \partial l^2_1 } &
    \sum_o \dfrac{ \partial L }{ \partial l^3_o } \dfrac{\partial l^3_o }{ \partial l^2_2 } &
    \cdots &
    \sum_o \dfrac{ \partial L }{ \partial l^3_o } \dfrac{\partial l^3_o }{ \partial l^2_{512} }
\end{bmatrix}
\end{aligned}

And so we can compute this with the matrix vector calculation:

$$
\dfrac{ \partial L }{ \partial \mathbf{l^2} } = \dfrac{ \partial L}{ \partial \mathbf{l^3}} (W^3)^T
$$ 


which is a $(1, 10)$ vector multiplying a $(10, 512)$ matrix. 

Now, we can finally compute $\dfrac{ \partial L}{\partial W^2} = \dfrac{\partial L}{\partial \mathbf{l^2} } \dfrac{\partial \mathbf{l^2}}{\partial W^2}$ which we can do again with the total derivaive! And we do it in exactly the same way we computed $\dfrac{ \partial L}{\partial W^3} = \dfrac{\partial L}{\partial \mathbf{l^3} } \dfrac{\partial \mathbf{l^3}}{\partial W^3}$ above. In this case it is the outer product of the vectors:


$$
\dfrac{ \partial L }{ \partial W^2 } = (\mathbf{l^1})^T \dfrac{ \partial L }{ \partial \mathbf{l^2} } 
$$


i.e. the outer product of a $(512, 1)$ vector by a $(1, 512)$ vector to give us a $(512, 512)$ matrix of derivatives. For example, the first row of $\dfrac{ \partial L }{ \partial W^2 }$ is how the loss changes with respect to a change in the weights that connect layer 1 unit $1$ to every unit in layer 2 unit. A change in the weight will change layer 2 unit $o$ by the value of layer 1 unit $1$ - this is the same for every weight in that row. This then is multiplied by how a small change in that unit $o$ in layer 2 effects the loss.

**It's worth reflecting on this. We have such complexity here. And it reduces really nicely.**

##### Computing the partial derivatives of $W^1$

Since we have done through all of the hard work of really inspecting what is going on under the hood, we can really simply the notation going forward. This compact notation really highlights the 'chain' aspect of the chain rule:

$$
\dfrac{\partial L}{\partial W^1} = \dfrac{\partial L}{\partial \mathbf{l^3}}  \dfrac{\partial \mathbf{l^3}}{\partial \mathbf{l^2}}  \dfrac{\partial \mathbf{l^2}}{\partial \mathbf{l^1}}  \dfrac{\partial \mathbf{l^1} }{\partial W^1}
$$

We are using the total derivative rule to condense all intermediate calculations, just like above:

First:

$$
\mathbf{g_1} = \dfrac{\partial L}{\partial \mathbf{l^2}} = \dfrac{\partial L}{\partial \mathbf{l^3}}  \dfrac{\partial \mathbf{l^3}}{\partial \mathbf{l^2}} = \dfrac{\partial L}{\partial \mathbf{l^3}} (W^3)^T
$$

$(1, 512) = (1, 10) \times (10, 512)$

$$
\mathbf{g_2} = \dfrac{\partial L}{\partial \mathbf{l^1}} =  \dfrac{\partial L}{\partial \mathbf{l^2}} \dfrac{\partial \mathbf{l^2}}{\partial \mathbf{l^1}} = \mathbf{g_1}(W^2)^T
$$

$(1, 512) = (1, 512) \times (512, 512)$ 

Finally we have the outer product calculation:

$$
\dfrac{\partial L}{\partial W^1} = \dfrac{\partial L}{\partial \mathbf{l^1}} \dfrac{\partial \mathbf{l^1}}{\partial W^1} = (\mathbf{x})^T \mathbf{g_2}
$$

$(512, 512) = (512, 1) \times (1, 512)$

It's awesome to see how changing the derivatives, backwards from the final layer, greatly simplifies computing the derivatives for our complex, fully connected network.

It is also very easy to compte, because we already have our weight vectors and we compute the output of each layer as part of our forward pass!

## The Code

In [ ]:
import numpy as np

run = False

class MyBasicNetwork:
    def __init__(self, learning_rate=0.02):

        self.a = learning_rate

        # Define weight matrix (output dim, input dim) by convention
        # Use zero-mean Xavier init (good for sigmoid, it has little
        # effect here as we don't use activation functions,
        # but useful for comparison.)
        self.W1 = np.random.randn(784, 512) * np.sqrt(1 / 784)
        self.W2 = np.random.randn(512, 512) * np.sqrt(1 / 512)
        self.W3 = np.random.randn(512, 10) * np.sqrt(1 / 512)

    def loss(self, l3, y):
        p = self.softmax(l3)[0][y]
        return -np.log( p + 1e-15)

    def softmax(self, vec):
        C = np.max(vec)
        return np.exp(vec - C) / np.sum(np.exp(vec - C))

    def predict(self, x):
        # forward pass through the network
        x = x.reshape(1, x.size)

        l1 = x @ self.W1
        l2 = l1 @ self.W2
        l3 = l2 @ self.W3

        pred = np.argmax(self.softmax(l3))

        return pred, l1, l2, l3

    def update_weights(self, x, y, verbose=False):

        _, l1, l2, l3 = self.predict(x)

        loss = self.loss(l3, y)

        if verbose:
            print(f"Loss: {loss}")

        # Compute the derivatives
        dloss_dl3 = self.softmax(l3)  # double check this
        dloss_dl3[0][y] -= 1

        dloss_dW3 = l2.T @ dloss_dl3       # (512, 10) = (512, 1) x (1, 10)

        dloss_dl2 = dloss_dl3 @ self.W3.T  # (1, 512) = (1, 10) x (10, 512)
        dloss_dW2 = l1.T @ dloss_dl2       # (512, 512) = (512, 1) x (1, 512)

        dloss_dl1 = dloss_dl2 @ self.W2.T  # (1, 512) = (1, 512) x (512, 512)
        dloss_dW1 = x.T @ dloss_dl1        # (784, 512) = (781, 1) x (1, 512)

        self.W3 -= self.a * dloss_dW3
        self.W2 -= self.a * dloss_dW2
        self.W1 -= self.a * dloss_dW1

# We won't run this here because it is very slow,
# but it gives an accuracy of ~73%
if run:
    # Initialise and train the model (no batching)
    model = MyBasicNetwork()


    for i, (X, y) in enumerate(training_data):

        x = np.asarray(X[0, :, :])
        y = int(y)

        model.update_weights(x, y)

        if i % 1000 == 0:
            print(f"Training iteration: sample: {i}")

    # Check the model accuracy
    results = np.empty(len(test_data))

    for i, (X, y) in enumerate(test_data):

        x = np.asarray(X[0, :, :])

        results[i] = model.predict(x)[0] == y

    print(f"Percent Correct: {np.mean(results) * 100}%")

## A Better Model

We will add some simple changes to improve the performance of the model. In our simple version about, we essentially had $\mathbf{l^3} = \mathbf{x}W$ where $W$ is a $(784, 10)$ matrix. One way to intepret this is a matrix encoding 10 linear hyperplanes in a $784$-dimension space that separate points according to their label.

A simple change we will make is to include a bias term $\mathbf{b}$. We can interpret this as adding an offset to each hyperplane, meaning our data does not need to be centered at zero.

More improtantly, we will also add an activation function around each unit. Once the activation at each unit is computed, we run it through a nonlinear function. Immediately, this means we loose the 'linear hyperplane' interpretation. This change now means the SEP BOUNDARY between groups do not need to be linear hyperplanes, but can be much more flexible and bendy. wNow, we will introduce an activation function which will make our network nonlinear. **LINK TO CS PAGE.**

Our network is now:

$$
\begin{aligned}
    l^1 &= \phi(\mathbf{x}W^1 + \mathbf{b^1}) \\
    l^2 &= \phi(\mathbf{l^1}W^2 + \mathbf{b^2}) \\
    l^3 &= \mathbf{l^2}W^3 + \mathbf{b^3} \\
\end{aligned}
$$

where $\phi(\cdot)$ is our nonlinear activation function. $\mathbf{b^i}$ is a $(1, n)$ vector of biases, one for each of $n$ units layer $i$. A note on the notation, the output of $\mathbf{l^{n-1}}W^n+\mathbf{b^n}$ in each layer is still a vector (e.g. $(1, 10)$) and we apply the nonlinear activation function element-wise to this vector.

Note we use the sigmoid function because it has interesting derivaives, but in general ReLu is preferred in larger, modern networks due to the [vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem). Note we don't apply the sigmoid function to the last layer, as we feed this directly to the cross-entropy loss which is already doing a mapping with the softmax function.

Now, we will take the derivatives of the loss with respect to the weights *and* biases. 

### Computing the derivatives

#### Derivatives of the sigmoid activation function

The sigmoid function maps the interval $(-\infty, \infty)$ to $(0, 1)$. Some intuition. 

$$
\phi(z(t)) = \dfrac{1}{1 + e^{-z(t)}}
$$

Let $z$ is some arbitrary function of the variable $t$. Then the derivative by the chain rule is:

$$
\dfrac{d}{dt} (1 + e^{-z(t)})^{-1} =  \dfrac{ e^{-z(t)} }{ (1 + e^{-z(t)})^2 } \dfrac{d}{dt} z(t)
$$

In our case, this will just include computing these extra terms of our layers next tot he derivative.

#### $W^3$ and $\mathbf{b^3}$

As we don't apply the sigmoid to the last layer, this is exactly the same as the simple example:

$$
\dfrac{\partial L}{\partial W^3} = (\mathbf{l^2})^T \dfrac{\partial L}{\partial \mathbf{l^3}}
$$

Note that the derivative of layer 3 with respect to the bias, $l^3_o = l^2_i W^3_{i, o} + b^3_o$ is $1$. This makes intuitive sense, when we make a small $\delta b^3_o$ change, it just changes the output of layer 3 exactly by this small change $\delta b^3_o$, because it is simply added to the output. Therefore:

$$
\dfrac{\partial L}{\partial \mathbf{b^3}} = \dfrac{\partial L}{\partial \mathbf{l^3}}
$$


#### $W^2$ and $\mathbf{b^2}$

Now things get tasty. Unfortunately, the notation gets even more complex, the easiest way to see what is going on is to follow along with a pen and paper, writing out each steps and converting to matrices as you go.

Recall, the same as above, a change in the loss with a change in the weights of layer 2 is:

$$
\dfrac{\partial L}{\partial W^2} = \dfrac{\partial L}{\partial \mathbf{l^3}}  \dfrac{\partial \mathbf{l^3}}{\partial \mathbf{l^2}}  \dfrac{\partial \mathbf{l^2} }{\partial W^2}
$$


##### $\dfrac{ \partial L }{ \partial \mathbf{l^2}}$

Because layer 3 does not have an activation function (and the derivative of the new bias term goes to zero) this is identical to the simple case above:

$$
\dfrac{ \partial L }{ \partial \mathbf{l^2}} = \dfrac{ \partial L }{ \partial \mathbf{l^3}} (W^3)^T
$$


##### $\dfrac{ \partial L }{ \partial W^2 }$

Now we are ready to compute the derivative of the loss with respect to the weight matrix of layer 2, and the bias vector. First starting with the weights, looking at each weight in isolation:

$$
\dfrac{ \partial L }{\partial W^2_{i, o}} = \dfrac{ \partial L }{ \partial l^2_o } \dfrac{ \partial l^2_o }{ \partial W^2_{i, o}}
$$

We calculated the first term above, so looking at the second term:

$$
\dfrac{ \partial l^2_o }{ \partial W^2_{i, o}} = \dfrac{ \partial }{ \partial W^2_{i, o}} \phi( \sum_n l^1_n W^2_{n, o} + b^2_o)
$$

Recalling the derivative of the term inside $\phi$ with respect to the weight $W^2_{i,o}$ is $l^1_i$ as it is $0$ when $n \neq i$. Letting $\hat{l^2_o} = \sum_i l^1_i W^2_{i, o} + b^2_o$:

$$
\dfrac{ \partial l^2_o }{ \partial W^2_{i,o}} = \dfrac{ e^{-\hat{l^2_o}} }{ (1 + e^{-\hat{l^2_o}})^2 } \ l^1_i
$$

We can implement this in matrix form. $\hat{l^2}$ is a $(1, 10)$ vector and so the element-wise multiplication is what we need here (officilally called the [Hadamard product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices)) with notation $\circ$. 

So in fact, the derivative looks very similar to before we had the activation function, except now we have this extra term related to its derivative:

$$
\begin{aligned}
    \dfrac{ \partial L }{\partial W^2} = (\mathbf{l^1})^T   \bigg(  \dfrac{ e^{-\mathbf{\hat{l^2}}} }{ (1 + e^{-\mathbf{\hat{l^2}}})^2 }   \circ \dfrac{ \partial L }{ \partial \mathbf{l^2} } \bigg)
\end{aligned}
$$

$$
(512, 512) = (512, 1) \times (1, 512) \circ (1, 512)
$$

and if we take that derivative with respect to $b_o$ then the term goes to $1$ and dissapears:

$$
    \dfrac{ \partial L }{\partial \mathbf{b^2}} =  \dfrac{ e^{-\mathbf{\hat{l^2}}} }{ (1 + e^{-\mathbf{\hat{l^2}}})^2 } \circ \dfrac{ \partial L }{ \partial \mathbf{l^2} }
$$


#### $W^1$ and $\mathbf{b^1}$

The derivation for layer 1 follows all the same ideas that we explored for layer 2.

$$
\dfrac{\partial L }{ \partial W^1} = \dfrac{\partial L }{ \partial \mathbf{l^3} } \dfrac{\partial \mathbf{l^3} }{ \partial \mathbf{l^2} } \dfrac{\partial \mathbf{l^2} }{ \partial \mathbf{l^1} } \dfrac{\partial \mathbf{l^1} }{ \partial \mathbf{W^1} }
$$

We just need to compute $\dfrac{\partial L }{ \partial \mathbf{l^1} }$ again, now we have the nonlinear activation function to deal with.

#### $\dfrac{\partial L}{ \partial \mathbf{l^1} }$ 

This will be a $(1, 512)$ vector where each element is how the loss changes with a small change in the corresponding unit of layer 1. For each unit in layer 1, we can compute this with the total derivative rule, by looking at the effect of each a small change in a layer 1 unit (input, $i$) on every unit in layer 2 (output, $o$) that it is connected to:

$$
\begin{aligned}
\dfrac{\partial L}{\partial l^1_i }  &= \sum_o \dfrac{\partial L}{\partial l^2_o } \dfrac{\partial l^2_o}{\partial l^1_i } 
\end{aligned}
$$

and note the output of a single layer 2 unit is:

$$
l^2_o = \phi \left( \sum_i l^1_i W^2_{i, o} + b^2_o \right)
$$

For the derivative of the term inside the activation function, this is exactly the same as the simple case. We take the derivative of $\sum_i l^1_i W^2_{i, o} + b^2_o$ with respect to a specific input unit $l^1_\hat{i}$ then all terms are zero except for the case where $i = \hat{i}$. Therefore, the derivative with respect to $l^1_\hat{i}$ is $W^2_{\hat{i}, o}$. 

Here, we will introduce the notation $\mathbf{\hat{l^2}} = \mathbf{l^1}W^2 + \mathbf{b^2}$, i.e. $\mathbf{\hat{l^2}}$ is the output of layer 2 before we put it through the activation function. 

Putting this together with the derivative of the activation function, as above:

$$
\begin{aligned}
\dfrac{\partial L}{\partial l^1_i } &= \sum_o \dfrac{\partial L}{\partial l^2_o } \dfrac{\partial l^2_o}{\partial l^1_i }  \\
 &= \sum_o \dfrac{\partial L}{\partial l^2_o }  \dfrac{\partial }{\partial l^1_i } \phi({\hat{l^2_o}} ) \\
&= \sum_o \dfrac{\partial L}{\partial l^2_o } \dfrac{ e^{-\hat{l^2_o}} }{ (1 + e^{-\hat{l^2_o}})^2 } W^2_{i, o}
\end{aligned}
$$

Intuitively, this makes sense. XXX. 

So we can implement this in vector form as:

$$
\dfrac{ \partial L }{ \partial \mathbf{l^1} } = \bigg( \dfrac{ \partial L }{ \partial \mathbf{l^2} }  \circ  \dfrac{ e^{-\mathbf{\hat{l^2}}} }{ (1 + e^{-\mathbf{\hat{l^2}}})^2 } \bigg) (W^2)^T
$$

The easiest way to see this vector form implementation is to write out all the terms with pen and paper and follow them through. MAKE CLEAR THE EPONENTIAL TERM IS A VECTOR AND WE APPLY THE EXPON INDIVIDUALLY.

#### So putting it all together for $W^1$ and $\mathbf{b^1}$

Again, we use $\hat{\mathbf{l^1}} = \mathbf{x}W^1 + \mathbf{b^1}$ for the output of layer 1 before inputting to the activation function. 

So as in the simple case, we will go through these piece-by-piece:

$$
\begin{aligned}
    \mathbf{g_1} &= \dfrac{\partial L }{ \partial \mathbf{l^2} } = \dfrac{\partial L }{ \partial \mathbf{l^3} } \dfrac{\partial \mathbf{l^3} }{ \partial \mathbf{l^2} }  \\
    &=  \dfrac{ \partial L }{ \partial \mathbf{l^3} } (W^3)^T
\end{aligned}
$$

$(1, 512) = (1, 10) \circ (1, 10) \times (10, 512)$ 

$$
\begin{aligned}
\mathbf{g_2} &= \dfrac{ \partial L }{ \partial \mathbf{l^1 }} \\ 
&= \mathbf{g_1} \dfrac{\partial \mathbf{l^2} }{ \partial \mathbf{l^1} } \\
&= \bigg( \mathbf{g_1} \circ \dfrac{ e^{-\hat{\mathbf{l^2}} } }{ (1 + e^{-\hat{\mathbf{l^2}}})^2 } \bigg) (W^2)^T
\end{aligned} 
$$

$(1, 512) = (1, 512) \circ (1, 512) \times (512, 512)$


$$
\begin{aligned}
\dfrac{\partial L}{ W^1} &= \mathbf{g_2} \dfrac{\partial \mathbf{l^1} }{ \partial W^1 }\\
&= \mathbf{x}^T \bigg( \dfrac{ e^{-\hat{\mathbf{l^1}} } }{ (1 + e^{-\hat{\mathbf{l^1}} })^2 } \circ \mathbf{g_2} \bigg)
\end{aligned}
$$

$$
(784, 512) = (784, 1) \times (1, 512) * (1, 512)
$$

When taking the derivative with respect to the bias, all steps are the same except the last step $\mathbf{x}$ is $1$ (to see this, take the derivative of $l^1_o = x_i W^1_{i, o} + b^1_o$ with respect to $b_0$ or $W^1_{i,o}$):

$$
\dfrac{ \partial L}{ \partial \mathbf{b^1}} = \dfrac{ e^{-\hat{\mathbf{l^1}} } }{ (1 + e^{-\hat{\mathbf{l^1}} })^2 } \circ \mathbf{g_2}
$$

TODO: add total derivative appendix
TODO: add some pictures of neurons
TODO: check all writing

In sum, thank god for autodiff!



In [ ]:

run = False

class MyBetterNetwork:
    def __init__(self, learning_rate=0.02):

        self.a = learning_rate

        # Define weight matrix (output dim, input dim) by convention
        # Use zero-mean Xavier init (good for sigmoid)
        # This makes a huge differences vs uniform.
        self.W1 = np.random.randn(784, 512) * np.sqrt(1 / 784)
        self.W2 = np.random.randn(512, 512) * np.sqrt(1 / 512)
        self.W3 = np.random.randn(512, 10) * np.sqrt(1 / 512)

        self.b1 = np.zeros((1, 512))
        self.b2 = np.zeros((1, 512))
        self.b3 = np.zeros((1, 10))

    def loss(self, l3, y):
        p = self.softmax(l3)[0][y]
        return -np.log( p + 1e-15)

    def softmax(self, vec):
        C = np.max(vec)
        return np.exp(vec - C) / np.sum(np.exp(vec - C))

    def predict(self, x):
        # forward pass through the network
        x = x.reshape(1, x.size)

        l1_hat = x @ self.W1 + self.b1
        l1 = self.phi(l1_hat)

        l2_hat = l1 @ self.W2 + self.b2
        l2 = self.phi(l2_hat)

        l3 = l2 @ self.W3 + self.b3

        pred = np.argmax(self.softmax(l3))

        return pred, l1_hat, l1, l2_hat, l2, l3

    def phi(self, vec):
        return 1 / (1 + np.exp(-vec))

    def dphi_dvec(self, vec):
        return np.exp(-vec) / (1 + np.exp(-vec))**2

    def update_weights(self, x, y, verbose=False):

        x = x.reshape(1, x.size)

        _, l1_hat, l1, l2_hat, l2, l3 = self.predict(x)

        loss = self.loss(l3, y)

        if verbose:
            print(f"Loss: {loss}")

        # Compute the derivatives
        dloss_dl3 = self.softmax(l3)  # double check this
        dloss_dl3[0][y] -= 1

        dloss_dW3 = l2.T @ dloss_dl3
        dloss_db3 = dloss_dl3

        dloss_dl2 = dloss_dl3 @ self.W3.T                               # (1, 512) = (1, 10) x (10, 512)
        dloss_dW2 = l1.T @ (self.dphi_dvec(l2_hat) * dloss_dl2)         # (512, 512) = (512, 1) x (1, 512) * (1, 512)
        dloss_db2 = self.dphi_dvec(l2_hat) * dloss_dl2                  # (1, 512) = (512, 1) x (1, 512)

        dloss_dl1 = (dloss_dl2 * self.dphi_dvec(l2_hat)) @ self.W2.T    # (1, 512) = (1, 512) * (1, 512) x (512, 512)
        dloss_dW1 = x.T @ (self.dphi_dvec(l1_hat) * dloss_dl1)          # (784, 512) = (784, 1) x (1, 512) * (1, 512)
        dloss_db1 = self.dphi_dvec(l1_hat) * dloss_dl1                  # (1, 512) = (1, 512) * (1, 512)

        self.W3 -= self.a * dloss_dW3
        self.W2 -= self.a * dloss_dW2
        self.W1 -= self.a * dloss_dW1

        self.b3 -= self.a * dloss_db3
        self.b2 -= self.a * dloss_db2
        self.b1 -= self.a * dloss_db1

# Initialise and train the model (no batching)
model = MyBetterNetwork()

for i, (X, y) in enumerate(training_data):

    x = np.asarray(X[0, :, :])
    y = int(y)

    model.update_weights(x, y, verbose=False)

    if i % 1000 == 0:
        print(f"Training iteration: sample: {i}")

# We won't run this here because it is very slow,
# but it gives an accuracy of ~83%
# Check the model accuracy
if run:
    results = np.empty(len(test_data))

    for i, (X, y) in enumerate(test_data):

        x = np.asarray(X[0, :, :])

        results[i] = model.predict(x)[0] == y

    print(f"Percent Correct: {np.mean(results) * 100}%")

**Appendix 1**

The cross entropy loss is:

\begin{aligned}
    L &= -\log \dfrac{ \exp{ l_{3, y} } }{ \sum_k \exp{ l_{3, k} } } \\
    &= -\bigg[ \log \exp{ l_{3, y} } - \log \sum_k \exp{ l_{3, k} } \bigg] \\
    &=  \log \sum_k \exp{ l_{3, k} } - l_{3, y}
\end{aligned}

(by the log laws). i.e. we take the logit  of layer 3 that matches the correct label $y$, normalise it to a probability
with the softmax function and take the negative log.

Let's start by taking the derivative with respect to $l_{3, y}$ where this is shorthand for $l_{3, i}$, $i=y$ i.e.
the layer 3 logit for the label that is correct for this image. We are asking: how does a small change in this logit effect the loss?

$$
\begin{aligned}
\dfrac{ \partial L }{ \partial l_{3, y} } &= \dfrac{ \partial }{ \partial l_{3, y} } \left( \log \sum_k \exp{ l_{3, k} } - l_{3, y} \right)
&=  \dfrac{ \partial }{ \partial l_{3, y} } \log \sum_k \exp{ l_{3, k} } - \dfrac{ \partial }{ \partial l_{3, y} }  l_{3, y}  \\
&= \dfrac{1}{ \sum_k \exp{ l_{3, k} } }  \dfrac{ \partial }{ \partial l_{3, y} } \sum_k \exp{ l_{3, k} } - 1
\end{aligned}
$$

(by the derivative of $\log x$ rule). Note that the last term will be $0$ when the the input dimension is not $y$ (because it is treated as a scalar).

We see that in the sum, the derivative of $\exp{ l_{3, i} } $ w.r.t $l_{3, k}$ is $\exp{ l_{3, i} }$ when $i = k$ and $0$ otherwise (as it is treated as a scalar).
So whatever dimension $i$ of $l_3$, we will input to the loss, we get $\text{softmax}(\mathbf{l_3})_i$ as the first term. But only when $i = y$ do we get $-1$ in the second term.

**Appendix 2**

In multivariate calculus, the total derivative rule captures how a function changes with respect to a variable that affects the function in multiple ways (through multiple intermediary functions).

For example, let's say we have the variable $t$ and let:

$$
w = f(x(t), y(t))
$$

The total derivative rule tells us how the output, $w$ changes with a small change in $t$. Intuitively, it is the sum of how $\delta t$ changes $w$ through $x(t)$ and how $\delta t$ changes $w$ through $y(t)$:

$$
\dfrac{dw}{dt} = \dfrac{\partial w}{ \partial x(t)} \dfrac{\partial x(t)}{ \partial t} + \dfrac{\partial w}{ \partial y(t)} \dfrac{\partial y(t)}{ \partial t} 
$$

Note this is exactly analogous to our set up with the layers and weights. A small change in a unit in layer 2 will effect the loss through every unit in layer 3. Let's look at unit 1 from layer 2 ($l^2_1$) as an example. Here layer 3 units are a function taking $l^2_1$, which is like $t$ in the above example, as an input:

$$
\text{loss} = L( \ l^3_1(l^2_1), \ l^3_2(l^2_1), \ ..., \ l^3_{10}(l^2_1) \ )
$$ 

$$
\dfrac{\partial L}{ \partial l^2_1 } = \dfrac{\partial L}{ \partial l^3_1 } \dfrac{ \partial l^3_1}{ \partial l^2_1 } + \dfrac{\partial L}{ \partial l^3_2 } \dfrac{\partial l^3_2 }{ \partial l^2_1 } + ... + \dfrac{\partial L}{ \partial l^3_{10} } \dfrac{\partial l^3_{10} }{ \partial l^2_1 }
$$

Which is exactly what we do to deal with these derivatives.